# Getting workspace

In [ ]:
from azureml.core import Workspace,Dataset,Environment,Experiment,ComputeTarget,Model
import json

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Creating environment

### Docs
[Environment class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment(class)?view=azure-ml-py) </br>
[How to docs](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments)  </br>



In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

#Defining inference environment
conda_deps = CondaDependencies.create(
    conda_packages=['numpy', 'xgboost'], 
    pip_packages=['azureml-defaults', 'inference-schema'],
    )
infenv = Environment(name='inference-environment')
infenv.python.conda_dependencies = conda_deps
infenv.register(ws)
print('Inference env registered')

# Deploying model
This example shows how to deploy a model to a Container Instance
### Docs
[How to](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=azcli) </br>
[Model class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py) </br>
[Model.Deploy() method](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#azureml-core-model-model-deploy)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
#Getting env
environment=Environment.get(ws,name='inference-environment')
#Creating inference configuration
inference_config = InferenceConfig(entry_script='./scripts/score.py', environment=environment)
#Auth_enables enables key based authentication against API.
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1,auth_enabled=True)

In [ ]:
service_name = 'diabetes-service'
#Getting model object
model=Model(ws,name='Diabetes-Model')
#Deploying service
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
#Output deploy logs to notebook
service.wait_for_deployment(show_output=True)

# Deploying to AKS cluster
In order to deploy to AKS cluster a simple config change is required. In this example a AKS cluster called 'mytestinference' is already created.
However this can also be created through the SDK.

```
#Deploy configuration
aks_config = AksWebservice.deploy_configuration()
#Fetching compute target
aks_target = ComputeTarget(ws,'mytestinference')
#Deploying service
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aks_config,
                       deployment_target=aks_target, #Pointing to AKS cluster
                       overwrite=True)
```


# Consuming model endpoint
Consuming the model endpoints can be done using the python SDK or as a regular REST API

In [ ]:
#Getting training data as example
training_data=Dataset.get_by_name(ws,'Diabetes_Processed')
df = training_data.to_pandas_dataframe()
#Removing label column
X = df.loc[:, df.columns != "Binary_Target"]

# Calling using AML SDK

In [ ]:
#Preparing json data
standard_sample_input = json.dumps({
    'data': X.iloc[157,:].to_dict()
    })

service.run(input_data=standard_sample_input)

# Calling as REST API
In AML go to the endpoints tab and navigate to your service, under the consume tab you will find the API keys and examples of how to consume the API. </br>
![image-alt-text](./img/aml-endpoint.bmp)

In [ ]:
import urllib
#Testing using https request
standard_sample_input = json.dumps({
    'data': X.iloc[1,:].to_dict()
    })
body = str.encode(standard_sample_input)

url = service.scoring_uri
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(json.loads(result))
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))